# AI Agents Course

This serios of notebooks is summarised and slightly adopted course [Introduction to LangGraph](https://academy.langchain.com/courses/intro-to-langgraph) by LangChain Academy.

## Setup
Best way to install is to use the `requirements.txt` file. In the root folder of the repository, run:
``` bash
python -m venv .venv
source .venv/bin/activate
pip install -r requirements.txt

```

# OPENAI_API_KEY 
Obtain your API key from https://platform.openai.com/api-keys. And set it as an environment variable:

``` bash
export OPENAI_API_KEY=<your-api-key>
```


## Chat models

In this course, we'll be using [Chat Models](https://python.langchain.com/v0.2/docs/concepts/#chat-models), which do a few things take a sequence of messages as inputs and return chat messages as outputs. LangChain does not host any Chat Models, rather we rely on third party integrations. [Here](https://python.langchain.com/v0.2/docs/integrations/chat/) is a list of 3rd party chat model integrations within LangChain! By default, the course will use [ChatOpenAI](https://python.langchain.com/v0.2/docs/integrations/chat/openai/) because it is both popular and performant.


Let's check that your `OPENAI_API_KEY` is set and, if not, you will be asked to enter it.

In [1]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

# Ok, we are set. Let's begin!

There are [a few standard parameters](https://python.langchain.com/v0.2/docs/concepts/#chat-models) that we can set with chat models. Two of the most common are:

* `model`: the name of the model
* `temperature`: the sampling temperature

`Temperature` controls the randomness or creativity of the model's output where low temperature (close to 0) is more deterministic and focused outputs. This is good for tasks requiring accuracy or factual responses. High temperature (close to 1) is good for creative tasks or generating varied responses. 

In [2]:
from langchain_openai import ChatOpenAI
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0)
gpt35_chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

Chat models in LangChain have a number of [default methods](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface). For the most part, we'll be using:

* `stream`: stream back chunks of the response
* `invoke`: call the chain on an input

And, as mentioned, chat models take [messages](https://python.langchain.com/v0.2/docs/concepts/#messages) as input. Messages have a role (that describes who is saying the message) and a content property. We'll be talking a lot more about this later, but here let's just show the basics.

In [3]:
from langchain_core.messages import HumanMessage
import json

# Create a message
msg = HumanMessage(content="Hello world", name="Lance")
# Q: What if we don't set the name? Why the name is needed?
#    Check what is the answer to the HumanMessage with content="What is my name?"
#    Provide a running example of a dialog that would be different if the name is not set.

# Message list
messages = [msg]

# Invoke the model with a list of messages 
response = gpt4o_chat.invoke(messages)
response

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_eb9dce56a8', 'finish_reason': 'stop', 'logprobs': None}, id='run-d3201767-237c-4530-b41c-8888ad8fb1de-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

We get an `AIMessage` response. Also, note that we can just invoke a chat model with a string. When a string is passed in as input, it is converted to a `HumanMessage` and then passed to the underlying model.


In [4]:
gpt4o_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_eb9dce56a8', 'finish_reason': 'stop', 'logprobs': None}, id='run-8c40c83d-cc75-4bfe-b09b-38f9f99a1561-0', usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
gpt35_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3a170093-dab1-4f0a-9f73-65ba0fcceb83-0', usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

The interface is consistent across all chat models and models are typically initialized once at the start up each notebooks. 

So, you can easily switch between models without changing the downstream code if you have strong preference for another provider.


## Messages

Chat models can use [`messages`](https://python.langchain.com/v0.2/docs/concepts/#messages), which capture different roles within a conversation.

LangChain supports various message types, including `HumanMessage`, `AIMessage`, `SystemMessage`, and `ToolMessage`.

These represent a message from the user, from chat model, for the chat model to instruct behavior, and from a tool call.

Let's create a list of messages.

Each message can be supplied with a few things:

* `content` - content of the message
* `name` - optionally, a message author
* `response_metadata` - optionally, a dict of metadata (e.g., often populated by model provider for `AIMessages`)

In [6]:
from langchain_core.messages import AIMessage, HumanMessage

messages = [AIMessage(content=f"So you said you were researching ocean mammals?", name="Model")]
messages.append(HumanMessage(content=f"Yes, that's right.", name="Lance"))
messages.append(AIMessage(content=f"Great, what would you like to learn about.", name="Model"))
messages.append(HumanMessage(content=f"I want to learn about the best place to see turtles in the Cyprus.", name="Lance"))

for m in messages:
    m.pretty_print()

================================== Ai Message ==================================
Name: Model

So you said you were researching ocean mammals?
================================ Human Message =================================
Name: Lance

Yes, that's right.
================================== Ai Message ==================================
Name: Model

Great, what would you like to learn about.
================================ Human Message =================================
Name: Lance

I want to learn about the best place to see turtles in the Cyprus.


In [7]:
result = gpt4o_chat.invoke(messages)
type(result)

langchain_core.messages.ai.AIMessage

In [8]:
print(result.content)

Cyprus is a fantastic place to observe sea turtles, particularly the loggerhead (Caretta caretta) and green turtles (Chelonia mydas). Here are some of the best places to see them:

1. **Lara Beach**: Located in the Akamas Peninsula, Lara Beach is one of the most important nesting sites for both loggerhead and green turtles in Cyprus. The beach is part of a protected area, and there are conservation efforts in place to ensure the safety of the turtles and their nests. Visiting during the nesting season, which typically runs from June to September, offers the best chance to see turtles.

2. **Alagadi Beach**: Situated near Kyrenia in Northern Cyprus, Alagadi Beach is another significant nesting site. The Society for the Protection of Turtles (SPOT) conducts conservation work here, and they often organize guided tours during the nesting season to educate visitors about the turtles and their conservation.

3. **Polis and Latchi Beaches**: These beaches, located in the Paphos District, are 

In [9]:
print(json.dumps(result.response_metadata, indent=2))

{
  "token_usage": {
    "completion_tokens": 342,
    "prompt_tokens": 66,
    "total_tokens": 408,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  },
  "model_name": "gpt-4o-2024-08-06",
  "system_fingerprint": "fp_f9f4fb6dbf",
  "finish_reason": "stop",
  "logprobs": null
}


# Task: 
1. Find the answer to all questions in the notebook (e.g. Q: what if we don't set the output type?)
2. There is also a SystemMessage. What is the difference between SystemMessage and HumanMessage?

   Provide an example of a dialog that would be different if we use SystemMessage instead of HumanMessage. 
   
   Use temperature=0 to make the model deterministic.

3. Can the dialog have more than one SystemMessage?